In [ ]:
import re

def titlesExt(soup):
    page_title = soup.find('title').get_text(strip=True) if soup.find('title') else 'No Title Found'
    title_column_data = {"Topic/Subject": page_title}

    return title_column_data

def imagesExt(soup):
    images = []
    for img_tag in soup.find_all('img'):
        img_src = img_tag.get('src')
        if img_src and img_src.startswith(('http://', 'https://')):
            images.append(img_src)
        elif img_src:
            # Handle relative URLs by trying to construct an absolute URL
            from urllib.parse import urljoin
            abs_img_src = urljoin(url, img_src)
            images.append(abs_img_src)

    images_data = [", ".join(images)]
    images_column_data = {"Images": images_data}

    return images_column_data

def locationExt(soup):
    pattern = r'<p[^>]*>(.*?)\s*•\s*</p>'
    match = re.search(pattern, soup)

    extracted_location = ""
    if match:
        extracted_location = match.group(1).strip()
    location_column_data = {"Location": extracted_location}

    return location_column_data


In [ ]:

def contentExt(soup):
    content_elements = soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'span', 'div', 'li', 'ol', 'ul'])

    extracted_texts = []
    seen_texts = set()

    for element in content_elements:
        if element.name in ['script', 'style', 'meta', 'link']: # Added more tags to skip
            continue

        text = element.get_text(strip=True)

        if text and text not in seen_texts and len(text) > 10: # Filter short, potentially irrelevant texts
            extracted_texts.append(text)
            seen_texts.add(text)

    article_content = "\n\n".join(extracted_texts)
    content_column_data = {"Collection": article_content}

    return content_column_data

In [ ]:
import re
import pandas as pd

def CategoriesExt(soup):
    regex_pattern = r'<span class="m_1e0e6180 mantine-Pill-label">(.*?)</span>'
    raw_extracted_contents = re.findall(regex_pattern, soup)

    # Clean up whitespace from each extracted item
    extracted_categories = [content.strip() for content in raw_extracted_contents]

    #print(f"Extracted Output: '{extracted_categories}'")
    Categories_column_data = {"Categories": extracted_categories}

    return Categories_column_data



#df = pd.DataFrame({'Categories': [extracted_contents_cleaned]})


In [ ]:

import re
from bs4 import BeautifulSoup

def extract_author_with_regex(soup_content):
    html_string = soup_content

    author_pattern = r'<div[^>]*class="[^"]*mantine-Stack-root[^"]*"[^>]*>.*?<p[^>]*>Author</p>.*?<a[^>]*href="[^"]*authorName=[^"]*"[^>]*>([^<]+)</a>'
    author_pattern_alt = r'<div[^>]*>.*?<p[^>]*>Author</p>.*?<span><a[^>]*href="[^"]*authorName=[^"]*"[^>]*>([^<]+)</a></span>'
    author_pattern_flexible = r'<p[^>]*>Author</p>.*?<a[^>]*href="[^"]*authorName=([^"&]+)"[^>]*>([^<]+)</a>'

    match = re.search(author_pattern, html_string, re.DOTALL | re.IGNORECASE)
    if match:
        extracted_author_name = match.group(1).strip()
        Author_column_data = {"Author": extracted_author_name}
        return Author_column_data

    match = re.search(author_pattern_alt, html_string, re.DOTALL | re.IGNORECASE)
    if match:
        extracted_author_name = match.group(1).strip()
        Author_column_data = {"Author": extracted_author_name}
        return Author_column_data

    match = re.search(author_pattern_flexible, html_string, re.DOTALL | re.IGNORECASE)
    if match:
        extracted_author_name = match.group(2).strip()
        Author_column_data = {"Author": extracted_author_name}
        return Author_column_data


    return ""



In [ ]:
def locationEXT(soup):
    regex_pattern = r'<p\s+class="mantine-focus-auto m_b6d8b162 mantine-Text-root"\s+style="[^"]*font-weight:\s*600;[^"]*">(.*?)</p>'

    # Search for the pattern in the provided whole_soup_content
    match = re.search(regex_pattern, soup, re.DOTALL)

    extracted_location = ""
    if match:
        full_content_from_tag = match.group(1).strip() # Remove outer leading/trailing whitespace

        extracted_location = full_content_from_tag.replace('•', '').strip()
    extracted_location = extracted_location.replace("<!-- -->","")
    location_column_data = {"Location": extracted_location}
    #print(f"\nResulting column: {location_column_data}")

    return location_column_data


In [ ]:
import requests
def main(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        soup = BeautifulSoup(response.text, 'html.parser')

        Title = titlesExt(soup)
        Images = imagesExt(soup)
        Collection =contentExt(soup)


        soup = str(soup)
        Location = locationEXT(soup) #locationExt(soup)
        Author = extract_author_with_regex(soup)  #AuthorName(soup) #Authorname(soup)
        Categories = CategoriesExt(soup)

        return Title,Location,Collection, Author,Categories,Images

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return "","","","","",""

if __name__ == "__main__":
    #url = "https://ruralindiaonline.org/article/my-mother-was-not-a-dayan" #"https://ruralindiaonline.org/article/beyond-disaster-and-wildlife-in-the-sundarbans"
    urls_excel_file = 'Rural_India_articles.xlsx'

    all_articles_dfs = []

    df_urls = pd.read_excel(urls_excel_file)

    for index, row in df_urls.iterrows():
        print(index,end=" ")
        article_url = row[0]

        if article_url:
            title, loc, collection,auth, categ, images  = main(article_url)
            combined_data = {}
            combined_data.update(title)
            combined_data.update(loc)
            combined_data.update(collection)
            combined_data.update({"Story link":article_url})
            combined_data.update(auth)
            combined_data.update(categ)
            combined_data.update(images)
            data_for_df = {key: [value] for key, value in combined_data.items()}

            df = pd.DataFrame(data_for_df)

            if not df.empty:
                all_articles_dfs.append(df)




0 

<ipython-input-16-3932525292>:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  article_url = row[0]


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [ ]:
merged_df = pd.concat(all_articles_dfs, ignore_index=True)
excel_file_name = "Extracted_article_dataF sample.xlsx"

merged_df.to_excel(excel_file_name, index=False)